#### The code is expanding the work available in: https://github.com/pucrs-automated-planning/pddl-parser , and; Specific snippets (for heuristics and informed (Dijkstra, A*, Greedy Best First) and uninformed (DFS, BFS) search algorithms) can be also mutated from: https://github.com/APLA-Toolbox/PythonPDDL

#### Check URL: https://github.com/remykarem/py2pddl for dynamic PDDL-based planning (Dynamic Python to Dynamic PDDL regeneration)

In [1]:
### Install a pip package (e.g., numpy) in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install numpy

## PDDL Parser (only):

In [2]:
## running the PDDL.py main file from the terminal using python
#!python PDDL.py # examples/dwr/dwr.pddl examples/dwr/pb1.pddl

## PDDL Planner (baseline BFS planner):

In [3]:
#!python -B planner.py examples/dinner/dinner.pddl examples/dinner/pb1.pddl -v

In [4]:
def pause(variable): # function to pause and print values in the console 
    print("********************************")
    print(f"Debug: \n{type(variable)}\n\n{variable}\n")
    wait = input("Press Enter to continue!")
    print("********************************")

In [5]:
def validate_conditions(state, positive, negative): # function to validate if positive and negative conditions are valid at a given state
        return positive.issubset(state) and negative.isdisjoint(state)

In [6]:
def state_transition(state, positive, negative): # function to apply the transition of the state (activate positive and deactivate negative effects)
        return state.difference(negative).union(positive)

In [7]:
def grounding_all_actions(parser): # Grounding all actions to generate every valid instantiation 
                                   # (based on the number and types of objects defined in problem.pddl)
    
    grounded_actions = []
    for action in parser.actions:
        for act in action.groundify(parser.objects, parser.types):
            # parser.objects are all instantiated objects (e.g., crane: [a, b, c, d])
            # parser.types are all different types of instantiated objects (e.g., crane)
            grounded_actions.append(act)
            
    return grounded_actions

In [8]:
def applicable_actions(state, grounded_actions): # Listing  all grounded (existing) actions that are applicable
                                                 # at the current state
    
    applicable_actions = []
    for act in grounded_actions:
        positive = act.positive_preconditions
        negative = act.negative_preconditions
    
        if validate_conditions(state, positive, negative):
            applicable_actions.append(act) 
    
    return applicable_actions

In [9]:
def solve_BFS(parser, grounded_actions):
        
        # Parsed data (all three objects are type: frozenset i.e., immutable static objects)
        state = parser.state # initial problem.pddl state
        goal_pos = parser.positive_goals # goal state positive conditions
        goal_not = parser.negative_goals # goal state negative conditions
        
        # Check if the goal state has been reached (no planning is required)
        if validate_conditions(state, goal_pos, goal_not):
            print('\nInitial state meets the goal conditions!')
            return []
                               
        # Graph Search
        closed_set = set([state]) # we already checked if the initial state is the goal state so we consider it in the closed_set
        frontier_set = [state, None] # frontier_set is a list of: [ state, (action that led to this state, plan that led to this state from the root) ]
        
        while frontier_set: # while frontier_set is not empty
            
            # implementing Breadth-First-Non-Informed-Search (BFS) where the visited/closed states set is served in FIFO manner
            # since the state under consideration is popped(0) which is the oldest of the appended 'frontier' set 'new_states'
            state = frontier_set.pop(0) # popping out (remove and assign) the first (or appended) state from the frontier_set
            plan = frontier_set.pop(0) # popping out (remove and assign) the first (or appended) plan from the frontier_set i.e., None
            
            for act in grounded_actions: # iterate over all grounded actions
                
                if validate_conditions(state, act.positive_preconditions, act.negative_preconditions): # check if the grounded action is currently applicable
                    new_state = state_transition(state, act.add_effects, act.del_effects)
                    
                    if new_state not in closed_set: # check if the new_state has not been visited/evaluated already
                        if validate_conditions(new_state, goal_pos, goal_not): # check if the new state is the goal state
                            
                            full_plan = [ act ] # initialise the full plan with the last action
                            while plan:
                                act, plan = plan # iteratively unfolding the enveloped plan and the respective sequence of actions
                                full_plan.insert(0, act) # populate the full_plan 0-position entry with the previous action i.e., the sequence of actions led to the goal state

                            return full_plan
                        
                        closed_set.add(new_state) # appending (at the end of the list) the new_state (which is not the goal state)
                        frontier_set.append(new_state) # appending (at the end of the list) the new_state (which is not the goal state)
                        frontier_set.append((act, plan)) # appending the grounded action led to the new_state alongside with the up-to-date plan(t) = [ action(t), plan(t-1) ]
                    
        return None # return 'None' if the goal state has not been reached

In [10]:
def export_plan(plan, verbose = True):
    
    print('\n----------------------------')
    if type(plan) is list:
        print('Plan:')
        for act in plan:
            print(act if verbose else act.name + ' ' + ' '.join(act.parameters))
    else:
        print('No plan was found')
        exit(1)

In [11]:
def export_parser(parser):
    
    print('\n----------------------------')
    print('Domain name: ' + parser.domain_name)
    for act in parser.actions:
        print(act)
    print('Problem name: ' + parser.problem_name)
    print('Objects: ' + str(parser.objects))
    print('Types: ' + str(parser.types))
    print('Initial State: ' + str(parser.state))
    print('Positive goal conditions: ' + str(parser.positive_goals))
    print('Negative goal conditions: ' + str(parser.negative_goals))

In [12]:
def generate_plan(parser, solver='BFS'):
    import time
    #export_parser(parser)
    
    start_planner_time = time.time()
    grounded_actions = grounding_all_actions(parser)

    # Planner instantiation
    if solver == 'BFS':
        plan = solve_BFS(parser, grounded_actions)
    elif solver == 'DFS':
        print('No DFS planner found!\nProceeding with BFS!')
        plan = solve_BFS(parser)
        
    plan_time = time.time()

    export_plan(plan)
    print(solver + ' Planner Time: ' + str(plan_time-start_planner_time) + 's\n')
    
    return plan, grounded_actions

In [13]:
def check_goal(parser, plan, step):
    # checking after new reached state (as the plan is being executed) whether goal has been achieved
    
    flag = False

    if validate_conditions(parser.state, parser.positive_goals, parser.negative_goals):
        flag = True
        print('\nReached state meets the goal state conditions:')
        print(parser.state)
        plan_length = len(plan)
        for counter in range(step+1,plan_length): # popping out (discarding) the last plan steps which
                                                  # were actually not executed to reach the goal
            plan.pop(-1)
            
    return plan, flag

#### Auxiliary functions enabling modifications in the parser objects and facts

In [14]:
def extract_parser_objects(parser): # extracting the objects defined/registered in the parser
    
    keys = parser.objects.keys()
    objects = []
    objects_set = set() # empty set

    for key in keys:
        temp_list = parser.objects.get(key)
        objects.extend(temp_list)
        objects_set = objects_set.union(set(temp_list))
        
    return keys, objects, objects_set

In [15]:
def objects_check(parser, items): # check whether an item is valid/true/exists in the declared objects set
    
    _, _, objects_set = extract_parser_objects(parser)
    # objects is a type of 'list-of-lists' and items is a value from a parser.objects.key()
    
    # just to randomly sample one already existing fact in the state and validate code
    # from random import sample
    # items_number = 1
    # items = sample(objects, items_number)[0]
    
    if set(items).intersection(objects_set) == set(items):
        flag = True # all items have been found in objects
    else:
        flag = False # not all items have been found in objects
    
    return flag

In [16]:
def objects_modify(parser, items, items_keys, mod_action): # modify a fact accordingly (discard or add) in the currently reached state
    
    keys, objects, objects_set = extract_parser_objects(parser)

    if mod_action == 'rem':
        if objects_check(parser, items): # if the items exist in the objects dictionary of the parser
            for item in items:
                for key in keys:
                    operation_rem = set(parser.objects.get(key)).difference(set([item]))                    
                    parser.objects[key] = list(operation_rem) # discard fact from objects dictionary

    elif mod_action == 'add':
        for item in items:
            for key in items_keys:
                if key in keys: # checking if the coupled key of the item exists in the domain
                    operation_add = set(parser.objects.get(key)).union(set([item]))
                    parser.objects[key] = list(operation_add) # add fact from objects dictionary
    
    return parser # returning the modified parser

#### Domain and Problem specific functions to support the definition of rationale facts

In [17]:
def find_item_location(parser, item, item_key):
    
    state = parser.state
    
    for fact in state:
        if item_key == 'robot':
            if ('at' in fact) and (item in fact):
                location = fact[2]
                
        if item_key == 'pile':
            if ('attached' in fact) and (item in fact):
                location = fact[2]
        
        if item_key == 'crane':
            if ('belong' in fact) and (item in fact):
                location = fact[2]
        
        if item_key == 'container':
            if ('in' in fact) and (item in fact):
                pile = fact[2]
                for _fact in state:
                    if ('attached' in _fact) and (pile in fact):
                        location = _fact[2]
                    
    return location

In [18]:
def find_robot_locations(parser):
    
    state = parser.state
    free_locations = []
    robot_locations = []
    
    for fact in state:
        if ('occupied' in fact):
            robot_location = fact[1]
            robot_locations.append(robot_location)
            
    free_locations = parser.objects.get('location').copy() # shallow copy of the locations list
    
    for robot_location in robot_locations:
        free_locations.remove(robot_location)
             
    return free_locations, robot_locations

In [19]:
def find_robot_facts_to_add(parser, item, item_key):
    # item = robot
    # item_key = 'robots'
    failure_alert = False

    from random import sample
    state = parser.state
    
    find_robot_facts_to_remove = []
    find_robot_facts_to_add = []
    
    free_locations, occupied_locations = find_robot_locations(parser)
    if free_locations:
        deployment_location = sample(free_locations, 1)[0]
    
        find_robot_facts_to_add.append(('occupied', deployment_location))
        find_robot_facts_to_add.append(('at', item, deployment_location))
        find_robot_facts_to_add.append(('unloaded', item))
    else:
        print('\nCannot deploy a new robot. All visible locations are occupied!')
        failure_alert = True
        
    return find_robot_facts_to_add, find_robot_facts_to_remove, failure_alert

In [20]:
def find_robot_facts_to_remove(parser, item, item_key):
    # item = robot
    # item_key = 'robots'
    
    from random import sample
    failure_alert = False
    state = parser.state
    
    location = find_item_location(parser, item, item_key)
    find_robot_facts_to_remove = []
    find_robot_facts_to_add = []

    for fact in state:
        if ('at' in fact) and (item in fact) and (location in fact): # unoccupying the location of the removed robot
            find_robot_facts_to_remove.append(fact)
            find_robot_facts_to_remove.append(('occupied', location))
    
    piles = parser.objects.get('pile')
    local_piles = []
    for fact in state: # collecting the local piles (attached to the location of the robot)
        for pile in piles:
            if ('attached' in fact) and (location in fact) and (pile in fact):
                local_piles.append(pile) # searching for the piles existing at the current location of the robot
    
    top_containers = []
    for fact in state: # collecting the top containers of the local piles
        for pile in local_piles:
            if ('top' in fact) and (pile in fact):
                top_container = fact[1]
                top_containers.append(top_container)
    
    for fact in state: # removing the unloaded robot fact (if it is unloaded)
        if ('unloaded' in fact) and (item in fact):
            find_robot_facts_to_remove.append(fact)
    
    for fact in state:
        if ('loaded' in fact) and (item in fact): # removing the loaded robot fact (if it is loaded)
            container = fact[2]
            find_robot_facts_to_remove.append(fact)
            
            selected_entry = sample(range(0,len(local_piles)), 1)[0] # select randomly the pile and the respective top container
            selected_pile = local_piles[selected_entry]
            top_container_of_selected_pile = top_containers[selected_entry]
            
            find_robot_facts_to_remove.append(('on', container, top_container_of_selected_pile)) # placing the container on top of 
                                                                   # the previous top container of the
                                                                   # randomly-selected pile at the current location 
                                                                   # of the robot
            
            find_robot_facts_to_remove.append(('top', container, selected_pile)) # placing the container on the top of a 
                                                                   # randomly-selected pile at the current location 
                                                                   # of the robot
            
    return find_robot_facts_to_add, find_robot_facts_to_remove, failure_alert

In [21]:
def find_location_facts_to_add(parser, item, key):
    # add new location as the robot(s) move closer to existing map's/locations' borders (to unveal new locations)
    # define a visibility value to add new locations (and probably new piles and new containers) accordingly
    from random import getrandbits
    failure_alert = False
    state = parser.state

    find_location_facts_to_add = []
    find_location_facts_to_remove = []
    
    existing_locations = parser.objects.get('location')
    for location in existing_locations: # adding a link with some (randomly) existing locations (to create a meshed topology)
        if bool(getrandbits(1)) or True: # randomly decide if each existing location will be linked with the new one
            find_location_facts_to_add.append(('adjacent', item, location))
            find_location_facts_to_add.append(('adjacent', location, item))
    
    # no need to add any other object fact: in case a crane, a pile, a robot or a new container needs to be deployed at this 
    # newly introduced location; they should be individually added using the respective find_"object"_fact_add functions.
    
    return find_location_facts_to_add, find_location_facts_to_remove, failure_alert

In [22]:
def find_location_facts_to_remove(parser, item, key):
    # remove existing location and every object located there!!!
    state = parser.state
    failure_alert = False
    find_location_facts_to_add = []
    find_location_facts_to_remove = []
    
    for fact in state:
        if ('at' in fact) and (item in fact): # if a robot is currently located at the same location spot
            robot = fact[1]
            new_facts, obsolete_facts = find_robot_facts_to_remove(parser, robot, 'robot')
            find_location_facts_to_add.extend(new_facts)
            find_location_facts_to_remove.extend(obsolete_facts)
            
        if ('belong' in fact) and (item in fact): # if a crane is currently located at the same location spot
            crane = fact[1]
            new_facts, obsolete_facts = find_crane_facts_to_remove(parser, crane, 'crane')
            find_location_facts_to_add.extend(new_facts)
            find_location_facts_to_remove.extend(obsolete_facts)
            
        if ('attached' in fact) and (item in fact): # if a pile is currently located at the same location spot
            pile = fact[1]
            new_facts, obsolete_facts = find_pile_facts_to_remove(parser, pile, 'pile')
            find_location_facts_to_add.extend(new_facts)
            find_location_facts_to_remove.extend(obsolete_facts)
        
        if ('adjacent' in fact) and (item in fact): # if a location is linked with other existing locations
            find_location_facts_to_remove.append(fact)

        # no need to remove the piled containers at this location since they are removed by removing their pile
            
    return find_location_facts_to_add, find_location_facts_to_remove, failure_alert

In [23]:
def find_crane_locations(parser):
    state = parser.state
    free_locations = []
    crane_locations = []
    
    for location in parser.objects.get('location'):
        for fact in state:
            if ('belong' in fact) and (location in fact):
                crane_locations.append(location)
    
    free_locations = parser.objects.get('location').copy() # shallow copy of the locations list
    
    for location in crane_locations:
        free_locations.remove(location)
    
    return free_locations, crane_locations

In [24]:
def find_crane_facts_to_add(parser, item, key):
    failure_alert = False
    find_crane_facts_to_add = []
    find_robot_facts_to_remove = []
    
    state = parser.state
    
    nocrane_locations, crane_locations = find_crane_locations(parser)
    if nocrane_locations:
        deployment_location = sample(nocrane_locations, 1)[0] # select randomly from the locations where no crane exists

        for fact in state:
            if ('belong' in fact) and (deployment_location in fact): # if the location the new pile was deployed at does not already have an existing crane
                find_crane_facts_to_add.append(('belong', 'crane', deployment_location)) # defining the crane deployment location
                find_crane_facts_to_add.append(('empty', 'crane')) # defining the initial crane status
                
    else:
        print('\nCannot deploy a new crane. Cranes exist in every location already!')
        failure_alert = True
    
    return find_crane_facts_to_add, find_robot_facts_to_remove, failure_alert

In [25]:
def find_crane_facts_to_remove(parser, item, key):
    # check if the crane is currently holding a container (if so then remove the crane and the container)
    find_crane_facts_to_remove = []
    find_crane_facts_to_add = []
    state = parser.state
    
    failure_alert = False
    
    for fact in state:
        if ('belong' in fact) and (item in fact):
            find_crane_facts_to_remove.append(fact)
            
        if ('empty' in fact) and (item in fact):
            find_crane_facts_to_remove.append(fact)
            
        if ('holding' in fact) and (item in fact):
            container = fact[2]
            find_crane_facts_to_remove.append(fact)
            
            to_add, to_remove, failure_alert_container = find_container_facts_to_remove(parser, container, 'container')
            if not failure_alert_container:
                find_crane_facts_to_remove.extend(to_remove)
                find_crane_facts_to_add.extend(to_add)
    
    return find_crane_facts_to_add, find_crane_facts_to_remove, failure_alert or failure_alert_container

In [26]:
def find_pile_facts_to_add(parser, item, key):
    # add new pile at a location (initializing its empty stack with a new 'pallet container')
    find_pile_facts_to_add = []
    find_pile_facts_to_remove = []
    failure_alert = False
    
    deployment_location = sample(parser.objects.get('location'),1)[0]
    find_pile_facts_to_add.append(('attached', item, deployment_location)) # defining the initial pile deployment location
    find_pile_facts_to_add.append(('top', 'pallet', item)) # defining the initial pile stack status

    nocrane_locations, crane_locations = find_crane_locations(parser)
    if nocrane_locations:
        if deployment_location in nocrane_locations: # if the pile is deployed in a location where there is no crane then add one
            find_pile_facts_to_add.append(('belong', 'crane', deployment_location)) # defining the crane deployment location
            find_pile_facts_to_add.append(('empty', 'crane')) # defining the initial crane status
        
    return find_pile_facts_to_add, find_pile_facts_to_remove, failure_alert

In [27]:
def find_pile_facts_to_remove(parser, item, key):
    
    # remove attached/existing pile from a location and every stacked container
    # (e.g., this pile has been removed/served by the logistics!)
    failure_alert = False

    find_pile_facts_to_remove = []
    find_pile_facts_to_add = []
    
    state = parser.state
    for fact in state:
        if ('attached' in fact) and (item in fact):
            location = fact[2]
            find_pile_facts_to_remove.append(('attached', item, location))
            
    find_pile_facts_to_remove.extend(find_piled_containers_facts(parser, item))

    return find_pile_facts_to_add, find_pile_facts_to_remove, failure_alert

In [28]:
def find_piled_containers_facts(parser, pile):
    
    find_piled_containers_facts = []
    
    state = parser.state
    piled_containers = []
    for fact in state:
        if ('in' in fact) and (pile in fact):
            container = fact[1]
            piled_containers.append(container)
            find_piled_containers_facts.append(('in', container, pile))
            find_piled_containers_facts.append(('equal', container, container))
            
        if ('top' in fact) and (pile in fact):
            find_piled_containers_facts.append(fact)
            
    for fact in state:
        for container in piled_containers:
            if ('on' in fact) and (container in fact):
                find_piled_containers_facts.append(fact)
        
    return find_piled_containers_facts

In [29]:
def find_container_facts_to_add(parser, item, key):
    
    # check the top containers of every pile
    # and put the specified container on top of a random selected_pile: ('top', item, selected_pile)
    # remove ('top', previous_top, selected_pile)
    # change the previously top container of the randomly selected pile appropriately: ('on', item, previous_top)
    # do not forget to define the facts:
    # ('equal', item, item)
    # ('in', item, selected_pile)
    failure_alert = False
    from random import sample
    
    find_container_facts_to_remove = []
    find_container_facts_to_add = []
    state = parser.state
    
    top_containers = []
    existing_piles = []
    for fact in state:
        if ('top' in fact):
            pile = fact[2]
            existing_piles.append(pile)
            
            top_container = fact[1]
            top_containers.append(top_container)
    
    if existing_piles: # if there any pile exists
        selected_entry = sample(range(0,len(existing_piles)),1)[0]
        selected_pile = existing_piles[selected_entry]
        selected_top_container = top_containers[selected_entry]
        
        find_container_facts_to_remove.append(('top', selected_top_container, selected_pile))
        
        find_container_facts_to_add.append(('top', item, selected_pile))
        find_container_facts_to_add.append(('in', item, selected_pile))
        find_container_facts_to_add.append(('on', item, selected_top_container))
        find_container_facts_to_add.append(('equal', item, item))
    
    return find_container_facts_to_add, find_container_facts_to_remove, failure_alert

In [30]:
def find_container_facts_to_remove(parser, item, key):
    failure_alert = False
    
    find_container_facts_to_remove = []
    find_container_facts_to_add = []
    state = parser.state
    
    # e.g., for cb container
    # ('equal', cb, cb)
    # ('in', cb, p1)
    # ('top', cb, p1)
    # ('on', cc, cb) # this translated to: the 'cc' container is on top of the 'cb' container
    # ('loaded', 'r1', 'cb') <-> ('unloaded', 'r1')
    # ('holding', 'k1', 'cb') <-> ('empty', 'k1')
    
    for fact in state:
        if ('equal' in fact) and (item in fact):
            find_container_facts_to_remove.append(fact)
            
        if ('in' in fact) and (item in fact):
            find_container_facts_to_remove.append(fact)
            
        if ('top' in fact) and (item in fact):
            pile = fact[2]
            find_container_facts_to_remove.append(fact)
            for _fact in state:
                if ('on' in _fact) and (item in _fact): # find the container right below the top (which is removed) one
                    below_container = _fact[2]
                    find_container_facts_to_add.append(('top',below_container,pile))
                    
        if ('on' in fact[0]) and (item in fact[1]): # the case where the item container is not on the top of a pile but somewhere
                                                    # in the middle of the pile (on top of another container)
            below_container = fact[2]
            find_container_facts_to_remove.append(fact)
            _state = list(set(state).difference(set(fact))) # temporarily excluding the first ('on', container, container) fact
            
            for _fact in _state:
                if ('on' in _fact[0]) and (item in _fact[2]):
                    top_container = _fact[1]
                    find_container_facts_to_remove.append(_fact)
                    find_container_facts_to_add.append(('on',top_container,below_container))
                    
        if ('loaded' in fact) and (item in fact): # the case where the item container is loaded on a robot
            robot = fact[1]
            find_container_facts_to_remove.append(fact)
            find_container_facts_to_add.append(('unloaded', robot))
            
        if ('holding' in fact) and (item in fact): # the case where the item container is loaded on a crane
            crane = fact[1]
            find_container_facts_to_remove.append(fact)
            find_container_facts_to_add.append(('empty', crane))
    
    return find_container_facts_to_add, find_container_facts_to_remove, failure_alert

In [31]:
def domain_facts_interpreter(parser, items, items_keys, mod_action): # this function is a domain specific function and allows to interpret
                                                    # the items/objects modifications to relevant facts to be incorporated
                                                    # in the parser.state as valid ones
    
    # e.g., when adding a new robot:
    # its location needs to be defined as: (at r1 l1);
    # as well as its load status: (unloaded r1)
    # facts in the parser.state frozenset are defined as tuples
    from random import sample
    
    new_facts = []
    obsolete_facts = []
    
    for key in items_keys:
        for item in items:
            if key == 'robot':
                if mod_action == 'add':
                    new_facts, obsolete_facts, failure_alert = find_robot_facts_to_add(parser, item, key)
                    
                elif mod_action == 'rem':
                    new_facts, obsolete_facts, failure_alert = find_robot_facts_to_remove(parser, item, key)
                    
            if key == 'location':
                if mod_action == 'add':
                    new_facts, obsolete_facts, failure_alert = find_location_facts_to_add(parser, item, key)
                    
                elif mod_action == 'rem':
                    new_facts, obsolete_facts, failure_alert = find_location_facts_to_remove(parser, item, key)
            
            if key == 'crane':
                if mod_action == 'add':
                    new_facts, obsolete_facts, failure_alert = find_crane_facts_to_add(parser, item, key)
                    
                elif mod_action == 'rem':
                    new_facts, obsolete_facts, failure_alert = find_crane_facts_to_remove(parser, item, key)
            
            if key == 'pile':
                if mod_action == 'add':
                    new_facts, obsolete_facts, failure_alert = find_pile_facts_to_add(parser, item, key)
                                        
                elif mod_action == 'rem':
                    new_facts, obsolete_facts, failure_alert = find_pile_facts_to_remove(parser, item, key)
                    
            if key == 'container':
                if mod_action == 'add':
                    new_facts, obsolete_facts, failure_alert = find_container_facts_to_add(parser, item, key)
                    
                elif mod_action == 'rem':
                    new_facts, obsolete_facts, failure_alert = find_container_facts_to_remove(parser, item, key)
    
    return new_facts, obsolete_facts, failure_alert

In [32]:
def facts_check(parser, facts): # check whether a fact is valid/true in the currently reached state
    
    state = parser.state
    # state is a type of 'frozenset' and fact is a type of triplet-'tuple'
    
    # just to randomly sample one already existing fact in the state and validate code
    # from random import sample
    # facts_number = 1
    # facts = sample(state, facts_number)[0]
    
    if facts in state:
        flag = True
    else:
        flag = False
                    
    return flag

In [33]:
def facts_modify(parser, new_facts, obsolete_facts, mod_frozenset): # modify a fact accordingly (discard or add) in the currently reached state
    
    if mod_frozenset == 'state':
        my_set = set(parser.state)
    elif mod_frozenset == 'goal':
        my_set = set(parser.goal)
    else:
        print('\nError! Non-existing parser set to modify!')
        return parser
    

    new_facts = set(new_facts)
    obsolete_facts = set(obsolete_facts)
    
    # state = parser.state # if you want to work with frozensets instead
    # facts = frozenset(facts) # if you want to work with frozensets instead
    
    
    #if facts_check(parser.state, facts): # check if the facts that need to be removed already exist
    my_set = my_set.difference(obsolete_facts) # discard facts from state frozenset
    
    # listoffrozensets = [state, facts] # if you want to work with frozensets instead
    # my_set = frozenset().union(*listoffrozensets) # if you want to work with frozensets instead
    my_set = my_set.union(new_facts) # add facts in state frozenset
    
    if mod_frozenset == 'state':
        # parser.state = state # if you want to work with frozensets instead
        parser.state = frozenset(my_set)
    elif mod_frozenset == 'goal':
        parser.goal = set(my_set)
    
    return parser

### Reward calculation framework

In [34]:
def find_locations_scores(parser, visibility=1):
    high_score = 5
    decay_rate = -1
    
    # find the distances from the occupied (reference -> visible) locations
    free_locations, robot_locations = find_robot_locations(parser)
    for robot_location in robot_locations:
        locations_distances = find_distance_from_reference_location(parser, [robot_location], visibility)
            
    # calculate the number of timesteps that each known (in the parser) location has been visited (+5 points for each visited location)
    # +4 points for each neighboring (adjacent) location of the ones visited (according to the visibility of the robots)
    # +3 points for each location within radius 2 of the ones visited etc.
    # at each timestep -1 point for all location points (losing their heat as time passes)
    
    # verify find_distance_from_reference_location() function
    # and use its outputs to calculate the coverage score for each robot
    # located in the occupied_locations!
    # locations_distance: is a dictionary where all locations are the keys and their respective values are
    # the distances from the one that is zeroed (in number of steps)
    # ???????????????????????
    # ???????????????????????
    # ???????????????????????
    
    locations_scores = {}
    total_locations_scores = 0
    for location_key, location_distance in locations_distances.items():
        location_score = high_score + location_distance*decay_rate
        locations_scores[location_key] = location_score
        total_locations_scores = total_locations_scores + location_score
    
    return locations_scores, total_locations_scores # this (dictionary, scalar) needs to be added to the previous 
                                                    # state correspoding dictionary values

In [35]:
def find_distance_from_reference_location(parser, reference_location, visibility):
    
    state = parser.state
    adjacent_facts = set()
    step = 0
    # if step == 0:
    locations_distances = populate_distance_dict(reference_location, step, {})# calculate the distance
                                                                            # between the reference_location
                                                                            # and itself = 0
    
    if visibility > 2:
        print('\nMaximum acceptable visibility: 2 steps!')
        visibility = min(visibility, 2)
    
    
    while step <= visibility:
        step += 1
 
        if step == 1: # calculating the distance between the reference_location and all of its adjacent ones
            adjacent_locations, adjacent_facts = find_adjacent_locations(reference_location, state.difference(adjacent_facts))
            locations_distances = populate_distance_dict(adjacent_locations, step, locations_distances)
            
        if adjacent_locations: # if there are any adjacent_locations with the originally reference_location then look for
                               # any adjacent_locations to the already adject_locations with the reference_location
            if step == 2:
                for reference_location in adjacent_locations:
                    adjacent_of_adjacent_locations, adjacent_facts = find_adjacent_locations(reference_location, state.difference(adjacent_facts))
                    locations_distances = populate_distance_dict(adjacent_of_adjacent_locations, step, locations_distances)
                
    return locations_distances

In [36]:
def populate_distance_dict(keys, step, locations_distances):
        
    for key in keys:
        locations_distances[key] = step
    
    return locations_distances

In [37]:
def find_adjacent_locations(reference_location, state):
    
    adjacent_locations = set()
    adjacent_facts = []
    
    for fact in state:

        if ('adjacent' in fact):

            if (fact[1] in reference_location): # find only the adjacent ones
                temp = set([fact[2]])
                adjacent_locations.update(temp)
                adjacent_facts.append(fact)

            elif (fact[2] in reference_location): # find only the adjacent ones
                temp = set([fact[1]])
                adjacent_locations.update(temp)
                adjacent_facts.append(fact)
    
    adjacent_locations = list(adjacent_locations)

    for fact in state:
        for adjacent_reference_location in adjacent_locations: # for each adjacent location check if it is adjacent
                                                               # with the other adjacent locations so as to remove
                                                               # it and not consider this relationship for the +1
                                                               # incremented visibility step adjacency check
            
            temp_adjacent_locations = adjacent_locations.copy() # shallow copy of adjacent_locations list to stop removing
                                                                # items from the original list but only from its shallow
                                                                # copy temp_adjacent_locations
            temp_adjacent_locations.remove(adjacent_reference_location)
            if ('adjacent' in fact) and (adjacent_reference_location in fact) and (any(location in temp_adjacent_locations for location in fact)):
                adjacent_facts.append(fact)
 
    adjacent_facts = set(adjacent_facts)

    return adjacent_locations, adjacent_facts

## Generating the initial plan

In [38]:
from PDDL import PDDL_Parser
import time

all_plans = []
domain = "examples/dwr/dwr.pddl"
problem = "examples/dwr/pb1.pddl"

start_time = time.time()
# Parser instantiation
parser = PDDL_Parser()
parser.parse_domain(domain)
parser.parse_problem(problem)
parse_time = time.time()
print('Parse Time: ' + str(parse_time-start_time) + 's\n')

# Generate plan
active_plan, grounded_actions = generate_plan(parser)
all_plans.append(active_plan)

Parse Time: 0.0023658275604248047s


----------------------------
Plan:
action: take
  parameters: ('k1', 'cc', 'cb', 'p1', 'l1')
  positive_preconditions: [['belong', 'k1', 'l1'], ['in', 'cc', 'p1'], ['on', 'cc', 'cb'], ['attached', 'p1', 'l1'], ['top', 'cc', 'p1'], ['empty', 'k1']]
  negative_preconditions: [['equal', 'cc', 'pallet'], ['equal', 'cc', 'cb']]
  add_effects: [['holding', 'k1', 'cc'], ['top', 'cb', 'p1']]
  del_effects: [['in', 'cc', 'p1'], ['top', 'cc', 'p1'], ['empty', 'k1'], ['on', 'cc', 'cb']]

action: load
  parameters: ('k1', 'r1', 'cc', 'l1')
  positive_preconditions: [['unloaded', 'r1'], ['belong', 'k1', 'l1'], ['at', 'r1', 'l1'], ['holding', 'k1', 'cc']]
  negative_preconditions: [['equal', 'cc', 'pallet']]
  add_effects: [['loaded', 'r1', 'cc'], ['empty', 'k1']]
  del_effects: [['unloaded', 'r1'], ['holding', 'k1', 'cc']]

action: move
  parameters: ('r1', 'l1', 'l2')
  positive_preconditions: [['at', 'r1', 'l1'], ['adjacent', 'l1', 'l2']]
  negative_preconditi

## Creating a dynamic re-planning framework

In [39]:
# function for advancing the system state by applying 'apply_plan_steps_to_replan' of the 'plan' action sequence
def progress_system_state(parser, plan, grounded_actions, simulation_step, apply_plan_steps_to_replan=1):    
    if apply_plan_steps_to_replan > len(plan):
        apply_plan_steps_to_replan = len(plan)

    if type(plan) is list:
        for step in range(0,apply_plan_steps_to_replan):
            act = plan[step] # applying the "optimal" action (according to the currently active_plan)
            # act = sample(applicable_actions(parser.state, grounded_actions),1)[0] # selecting a random (applicable) action
            
            simulation_step += 1
            
            print('\n**************************')
            print(f'Simulation Step: {simulation_step}')
            print('**************************\n')
            print('Applying Planned Action:')
            print(act)
            new_state = state_transition(parser.state, act.add_effects, act.del_effects)
            parser.state = new_state
            plan, goal_flag = check_goal(parser, plan, step)
            
            if goal_flag:
                return parser, simulation_step, plan
        
        print('\nReached state:')
        print(parser.state)
        
    return parser, simulation_step, []

In [40]:
from random import sample

apply_steps_until_next_replan = 1 # e.g., apply applied_plan_steps_to_replan=1 step(s) of the currently active plan 
                                  # and then replan considering as an initial state the current state
number_of_replanning_attempts = 100 # e.g., number of attempts to replan after 'apply_plan_steps_to_replan'
simulation_step = 0                 # initial value for the total simulation step

new_change_applied = 0            # flag that indicates a changes has been imposed at the current timestep (compared to previous timestep)
number_of_replanning_attempts = min(number_of_replanning_attempts, len(active_plan)) # making sure than the replanning 
                                                                    # attempts are always less than the initial plans' length

locations_scores = populate_distance_dict(parser.objects.get('location'), 0, {}) # initializing locations_scores with zeros

for replans in range(1,number_of_replanning_attempts):
    
    if active_plan:  # if active_plan is not empty
        
        parser, simulation_step, active_plan_head_used = progress_system_state(parser, active_plan, grounded_actions, simulation_step, apply_steps_until_next_replan)
        # parser, new_change_applied = impose_stochastic_disturbance(parser, simulation_step, type_of_disturbance='none')

        # ----------- Adding a new location at timestep = 7 ----------- #
        if simulation_step == 7:
            
            new_change_applied = True            
            mod_action = 'add'
            mod_frozenset = 'state'
            items_keys = ['location']
            items = ['l3']
            
            print('\nApplying Modification:')
            print(f'set:{mod_frozenset} - action:{mod_action} - keys:{items_keys} - items:{items}')
            
            # e.g., Add all relevant facts (depends on the domain) for the added items in the parser.objects dictionary
            # facts = sample(parser.state, 1)[0] # randomly selected fact from the already existing ones just for debugging
            new_facts, obsolete_facts, failure_alert = domain_facts_interpreter(parser, items, items_keys, mod_action)
            
            if not failure_alert:
                parser = facts_modify(parser, new_facts, obsolete_facts, mod_frozenset)
                # e.g., Add or remove items in the current objects dictionary of the parser
                objects_modify(parser, items, items_keys, mod_action)

        else:
            new_change_applied = False
        # ----------- Adding a new location at timestep = 7 ----------- #
            
        # ----------- Removing the added location at timestep = 10 ----------- #
        if simulation_step == 10:

            new_change_applied = True            
            mod_action = 'rem'
            mod_frozenset = 'state'
            items_keys = ['location']
            items = ['l3']

            print('\nApplying Modification:')
            print(f'set:{mod_frozenset} - action:{mod_action} - keys:{items_keys} - items:{items}')

            # e.g., Add all relevant facts (depends on the domain) for the added items in the parser.objects dictionary
            # facts = sample(parser.state, 1)[0] # randomly selected fact from the already existing ones just for debugging
            new_facts, obsolete_facts, failure_alert = domain_facts_interpreter(parser, items, items_keys, mod_action)

            if not failure_alert:
                parser = facts_modify(parser, new_facts, obsolete_facts, mod_frozenset)
                # e.g., Add or remove items in the current objects dictionary of the parser
                objects_modify(parser, items, items_keys, mod_action)

        else:
            new_change_applied = False
        # ----------- Removing the added location at timestep = 10 ----------- #
        
        # ----------- Calculating surveillance score at the current timestep ----------- #
        current_locations_scores, total_locations_scores = find_locations_scores(parser, visibility=1)
        location_keys = locations_scores.keys()
        for current_location_key, current_location_score in current_locations_scores.items():
            if current_location_key in location_keys: # if the location key already exists in the initiliazed
                                                      # location_scores dictionary then update its value
                locations_scores[current_location_key] += current_locations_scores[current_location_key]
            else: # if the location key does not already exist in the initiliazed
                  # location_scores dictionary then set its value
                locations_scores[current_location_key] = current_locations_scores[current_location_key]
            
        print(f'\nAccumulated surveillance score for each location: {locations_scores}')
        print(f'\nTotal current surveillance score: {total_locations_scores}')
        # ----------- Calculating surveillance score at the current timestep ----------- #
        
        if active_plan_head_used: # if active_plan_tail is not empty then goal state has been unexpectedly reached before finishing the plan
            all_plans.pop(-1) # removing the entire plan that was most recently generated
            all_plans.append(active_plan_head_used) # to replace it with its head/fraction that was actually used to reach goal state
            break
        
        #print(parser.objects.get(items_keys[0]))
        #print(parser.state)
        
        print('Replanning attempt...')
        
        # replanning after applying actions from the previously generated plan
        if new_change_applied:
            active_plan, grounded_actions = generate_plan(parser)
            all_plans.append(active_plan) # replanning after applying actions based on the previously generated plan
        else:
            active_plan.pop(0)   


**************************
Simulation Step: 1
**************************

Applying Planned Action:
action: take
  parameters: ('k1', 'cc', 'cb', 'p1', 'l1')
  positive_preconditions: [['belong', 'k1', 'l1'], ['in', 'cc', 'p1'], ['on', 'cc', 'cb'], ['attached', 'p1', 'l1'], ['top', 'cc', 'p1'], ['empty', 'k1']]
  negative_preconditions: [['equal', 'cc', 'pallet'], ['equal', 'cc', 'cb']]
  add_effects: [['holding', 'k1', 'cc'], ['top', 'cb', 'p1']]
  del_effects: [['in', 'cc', 'p1'], ['top', 'cc', 'p1'], ['empty', 'k1'], ['on', 'cc', 'cb']]


Reached state:
frozenset({('adjacent', 'l1', 'l2'), ('attached', 'p2', 'l2'), ('at', 'r1', 'l1'), ('in', 'cd', 'q1'), ('unloaded', 'r1'), ('in', 'ca', 'p1'), ('on', 'cb', 'ca'), ('top', 'pallet', 'q2'), ('attached', 'q2', 'l2'), ('in', 'cf', 'q1'), ('equal', 'cf', 'cf'), ('top', 'cb', 'p1'), ('equal', 'cd', 'cd'), ('on', 'cd', 'pallet'), ('top', 'cf', 'q1'), ('belong', 'k1', 'l1'), ('adjacent', 'l2', 'l1'), ('equal', 'pallet', 'pallet'), ('equal', 